In [ ]:
%load_ext line_profiler
%load_ext autoreload

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from scipy.special import expit as sigmoid
from scipy.stats import norm
from scipy import integrate

import tqdm.notebook as tqdm

In [ ]:
P = 5000
d = 1

# mu = np.random.randn(d)
# Sigma = np.random.randn(d,d)
# Sigma = Sigma @ Sigma.T / d  

mu = np.zeros(d)
Sigma = np.eye(d)

x = np.random.multivariate_normal(mu, Sigma, P)

a0 = np.random.randn(d)
a0 = np.sqrt(d) * a0 / np.linalg.norm(a0)

In [ ]:
c = 3 #inverse std of logistic noise
y = np.random.binomial(1, sigmoid(c * x @ a0 / np.sqrt(d)))*2 - 1 #scaled heaviside of logistic noise

In [ ]:
V = y[:,None] * x@a0 / np.sqrt(d)

In [ ]:
lamb = 1e-6
eta = lambda m: np.maximum(0, 1-m)
H = lambda a: np.mean(eta(y[:,None] * x @ a/np.sqrt(d)), 0) + lamb*(a**2).sum(0)

In [ ]:
grid = np.linspace(-0,2,100).reshape(1, -1)
plt.plot(grid[0,:], H(grid))
plt.axvline(a0)

In [ ]:
grid

In [ ]:
if d == 1:
    h1 = plt.hist(x[np.sign(V-1) == 1], density=True, bins=100, alpha=.8)
    h2 = plt.hist(x[np.sign(V-1) == -1], density=True, bins=100, alpha=.8)

In [ ]:
loc = a0 @ mu / np.sqrt(d)
scale = np.sqrt(a0.T @ Sigma @ a0 / d)
p = lambda x: sigmoid(c * x) * (norm.pdf(x, loc=loc, scale=scale) + norm.pdf(x, loc=-loc, scale=scale))

In [ ]:
hist = plt.hist(V, bins=200, density=True, label='Simulated')

grid = np.linspace(min(V),max(V))
plt.plot(grid, p(grid), lw=5, label='Analytic')

plt.legend()
plt.title(r'PDF of $V = y a_0^T x / \sqrt{d}$')

In [ ]:
from scipy.stats import rv_continuous

In [ ]:
class VDistribution(rv_continuous):
    def __init__(self, loc, scale, c, name='VDistribution'):
        super().__init__(a=-np.inf, b=np.inf)
        
        self.loc = loc
        self.scale = scale
        self.c = c
        
    def _pdf(self, x):
        return sigmoid(self.c * x) * (norm.pdf(x, loc=self.loc, scale=self.scale) + norm.pdf(x, loc=-self.loc, scale=self.scale))

In [ ]:
V = VDistribution(loc, scale, c)
Z = norm(0, 1)

In [ ]:
def V_pdf(x):
    return sigmoid(self.c * x) * (norm.pdf(x, loc=self.loc, scale=self.scale) + norm.pdf(x, loc=-self.loc, scale=self.scale))

In [ ]:
from scipy import integrate
from numpy import inf

In [ ]:
def pVZ(v, z):
    V = VDistribution(loc, scale, c)
    Z = norm(0, 1)
    return Z.pdf(z)*V.pdf(v)

In [ ]:
integrate.dblquad(lambda z,v: Z.pdf(z)*V.pdf(v), -inf, inf, lambda a: -inf, lambda b: inf)

In [ ]:
integrate.romberg(V.pdf, -inf, inf)

In [ ]:
def f1(alpha, sigma, gamma):
    """
        P(αV +σZ ≤ 1−γ)
    """
    return V.expect(lambda v: Z.cdf((1-gamma)/sigma - alpha/sigma * v))

def f2(alpha, sigma):
    """
        P(αV +σZ ≥1)
    """
    return V.expect(lambda v: 1 - Z.cdf(1/sigma - alpha/sigma * v))

def f3(alpha, sigma, gamma):
    """
        E[ ((1 − (αV + σZ)) / γ)^2  * I[1−γ ≤ αV +σZ ≤ 1] ]
    """
    return V.expect(
        lambda v: Z.expect(
            lambda z: ((1 - (alpha*v + sigma*z)) / gamma)**2, lb = (1-gamma-alpha*v)/sigma, ub = (1-alpha*v)/sigma
        )
    )

def g1(lamb, gamma, delta):
    return (2*lamb*gamma-1)*delta+1

def g2(sigma, gamma, delta):
    return delta*(sigma/gamma)**2


In [ ]:
def eq1(alpha, sigma, gamma, delta, lamb):
    return f1(alpha, sigma, gamma) + f2(alpha, sigma) - g1(lamb, gamma, delta)

def eq2(alpha, sigma, gamma, delta, lamb):
    return f1(alpha, sigma, gamma) + f3(alpha, sigma, gamma) - g2(sigma, gamma, delta)

In [ ]:
def train_loss(alpha, sigma, gamma):
    return V.expect(
        lambda v: Z.expect(
            lambda z: 1 - gamma - alpha*v - sigma*z, ub=(1-gamma-alpha*v)/sigma
        )
    ) 

def test_loss(alpha, sigma):
    """
        The paper gives no expression for this. This is a guess.
    """
    return V.expect(
        lambda v: Z.expect(
            lambda z: 1 - alpha*v - sigma * z, ub=(1-alpha*v)/sigma
        )
    ) 

In [ ]:
from scipy import optimize

In [ ]:
cons = [{'type':'eq', 'fun': lambda p: eq1(*p, delta, lamb)}, {'type':'eq', 'fun': lambda p: eq2(*p, delta, lamb)}]

In [ ]:
def obj(p):
    alpha, sigma, gamma = p
    
    return V.expect(
        lambda v: Z.expect(
            lambda z: 1 - gamma - alpha*v - sigma * z, ub=(1-gamma-alpha*v)/sigma
        )
    ) + lamb*delta*(sigma**2 + alpha**2)

In [ ]:
lamb = 1e-6
delta = 1

gamma = .42
sigma = .40
alpha = .28

res = optimize.minimize(obj, (alpha, sigma, gamma), constraints=cons, options={'disp': True, 'iprint':5})

In [ ]:
res = optimize.minimize(obj, (alpha, sigma, gamma), constraints=cons, options={'disp': True, 'iprint':5})

In [ ]:
alpha, sigma, gamma = res.x

In [ ]:
res

In [ ]:
train_loss(alpha, sigma, gamma)

In [ ]:
test_loss(alpha, sigma)

In [ ]:
sigma

In [ ]:
gamma

In [ ]:
1 - gamma + alpha*1 + sigma * 1

In [ ]:
lamb*delta*(sigma**2 + alpha**2)

## Loop

In [ ]:
lamb = 1e-6

deltas = np.linspace(.1, 2, 19)
deltas = np.concatenate((deltas, [1.0]))

results = []

for delta in tqdm.tqdm(deltas):
    
    #initial values
    gamma = 1
    sigma = 1
    alpha = 1

    def obj(p):
        alpha, sigma, gamma = p

        return V.expect(
            lambda v: Z.expect(
                lambda z: 1 - gamma - alpha*v - sigma * z, ub=(1-gamma-alpha*v)/sigma
            )
        ) + lamb*delta*(sigma**2 + alpha**2)
    
    cons = [{'type':'eq', 'fun': lambda p: eq1(*p, delta, lamb)}, {'type':'eq', 'fun': lambda p: eq2(*p, delta, lamb)}]
    out = optimize.minimize(obj, (alpha, sigma, gamma), constraints=cons, bounds = [(0, np.inf)]*3)
    alpha, sigma, gamma = out.x
    
    tr_ls = train_loss(alpha, sigma, gamma)
    te_ls = test_loss(alpha, sigma)
    

    result = {
        'lambda':lamb,
        'delta': delta,
        'gamma': gamma,
        'sigma': sigma,
        'alpha': alpha,
        'v_loc': loc,
        'v_scale': scale,
        'v_c': c,
        'train_loss': tr_ls,
        'test_loss': te_ls
    }
    results.append(result)
    
    print(result)

In [ ]:
import pandas as pd

In [ ]:
result_df = pd.read_json(open('results/SVM_theoretical_gaussian_cov.json', 'r'))

In [ ]:
# result_df = pd.DataFrame(results)

In [ ]:
# result_df.to_json(open('results/SVM_theoretical_gaussian_cov.json', 'w'))

In [ ]:
result_df

In [ ]:
plt.plot(result_df.delta, np.log(result_df.alpha))

In [ ]:
plt.plot(result_df.delta, np.log(result_df.test_loss))
# plt.plot(result_df.delta, result_df.train_loss)